# Clean database tables

Going to remove any duplicated replay_url's from league_matches and jobs tables.

## Table of Contents

1. [Clean tables](#clean-tables)
    - [Imports](#imports)
    - [Configurations](#configurations)
    - [Query all data from league_matches](#query-all-data-from-league_matches)
    - [Find all duplicated replay_urls](#find-all-duplicated-replay_urls)
    - [Go through each to find incorrect match data](#go-through-each-to-find-incorrect-match-data)
    - [Collect league_matches id's to remove](#collect-league_matches-ids-to-remove)
1. [Remove duplicated ids from database](#remove-duplicated-ids-from-database)
    - [Imports](#imports-1)
    - [Remove ids from league_matches and jobs tables](#remove-ids-from-league_matches-and-jobs-tables)

## Clean tables

### Imports

In [1]:
import polars as pl

from gfwldata.utils.db import sync_engine

### Configurations

In [ ]:
# Don't cut off column length configuration
pl.Config(fmt_str_lengths=100)

### Query all data from league_matches

In [68]:
raw_tbl = pl.read_database(query="select * from league_matches", connection=sync_engine)

### Find all duplicated replay_urls

In [33]:
with pl.Config(fmt_str_lengths=100):
    replay_url_duplicates = (
        raw_tbl.filter(pl.col("replay_url").is_not_null())
        .group_by("replay_url")
        .len()
        .filter(pl.col("len") > 1)
    )

    print(replay_url_duplicates)

shape: (4, 2)
┌────────────────────────────────────────────┬─────┐
│ replay_url                                 ┆ len │
│ ---                                        ┆ --- │
│ str                                        ┆ u32 │
╞════════════════════════════════════════════╪═════╡
│ https://duelingbook.com/replay?id=68321903 ┆ 2   │
│ https://duelingbook.com/replay?id=67849343 ┆ 2   │
│ https://duelingbook.com/replay?id=67778738 ┆ 2   │
│ https://duelingbook.com/replay?id=55940660 ┆ 2   │
└────────────────────────────────────────────┴─────┘


### Go through each to find incorrect match data

In [59]:
(
    raw_tbl.filter(
        pl.col("replay_url").is_in(replay_url_duplicates["replay_url"].to_list())
    )
    .select(pl.exclude(["replay_id", "created_at", "updated_at"]))
    .sort(["replay_url"])
)

season,week,team1,team2,team1_player,team2_player,team1_player_deck_type,team2_player_deck_type,match_score,replay_url,id
i64,i64,str,str,str,str,str,str,str,str,str
4,3,"""The Worst Generation""","""Rehab""","""don't copy""","""insids""","""Chaos Turbo""","""Chaos Turbo""","""2-0""","""https://duelingbook.com/replay?id=55940660""","""bfda413aac224bcaabd7a9ecb8fb7d3f"""
4,3,"""Masters of Oz""","""Purple Haze""","""jase""","""raysaber""","""Chaos Turbo""","""Chaos Turbo""","""1-2""","""https://duelingbook.com/replay?id=55940660""","""d6d4c00c5f06465b9d0ce393e8992ca3"""
6,4,"""team_bomb_squad""","""team_batl_oxen""","""The dark knight""","""psyt""","""Chaos Turbo""","""Cyber-Stein OTK""","""2-1""","""https://duelingbook.com/replay?id=67778738""","""63e653b80fee48728ba4c0a8debf5834"""
6,4,"""team_batl_oxen""","""team_bomb_squad""","""psyt""","""The dark knight""","""Cyber-Stein OTK""","""Chaos Turbo""","""2-0""","""https://duelingbook.com/replay?id=67778738""","""e230d51d0a604b21939244d47a4ee3df"""
6,4,"""team_black_luster_serbs""","""team_4hunnids""","""popz""","""Lukaz""","""Reasoning Gate Turbo""","""Chaos Turbo""","""2-1""","""https://duelingbook.com/replay?id=67849343""","""d3054c2c57d048939849a2e85e1c7ad2"""
6,4,"""team_black_luster_serbs""","""team_4hunnids""","""popz""","""SolMasterMatt""","""Reasoning Gate Turbo""","""Chaos Turbo""","""2-1""","""https://duelingbook.com/replay?id=67849343""","""f596b7ca06c0460591b2e9ee2d73e30d"""
6,6,"""team_the_bones_generation""","""team_carbombnara""","""Shifty""","""CCaliendo""","""Chaos Turbo""","""Chaos Control""","""2-1""","""https://duelingbook.com/replay?id=68321903""","""19226427495a400a99792badbb63fe13"""
6,6,"""team_the_bones_generation""","""team_carbombnara""","""Shifty""","""marcor96""","""Chaos Turbo""","""Chaos Control""","""2-1""","""https://duelingbook.com/replay?id=68321903""","""5c25d683cdbd449eaaa9e1ea2e9e155a"""


### Collect league_matches id's to remove

In [67]:
incorrect_matches_id = [
    "bfda413aac224bcaabd7a9ecb8fb7d3f",
    "e230d51d0a604b21939244d47a4ee3df",
    "d3054c2c57d048939849a2e85e1c7ad2",
    "19226427495a400a99792badbb63fe13",
]

## Remove duplicated ids from database

### Imports

In [73]:
from uuid import UUID

from sqlalchemy import delete

from gfwldata.utils.db import get_db_session
from gfwldata.utils.models import Job, LeagueMatch

### Remove ids from league_matches and jobs tables

In [81]:
# Convert string of ids to uuid
league_ids_to_delete = [UUID(id) for id in incorrect_matches_id]

# Delete duplicated ids from league_matches table
with get_db_session() as session:
    stmt = delete(LeagueMatch).where(LeagueMatch.id.in_(league_ids_to_delete))
    result = session.execute(stmt)

    session.commit()

# Delete duplicated ids from jobs table
with get_db_session() as session:
    stmt = delete(Job).where(Job.league_match_id.in_(league_ids_to_delete))
    result = session.execute(stmt)

    session.commit()